In [1]:
!pip install pyspark

In [2]:
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [3]:
import numpy as np
import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from matplotlib import rcParams
import matplotlib as mpl
%matplotlib inline
import seaborn as sns

colorset10 = sns.cubehelix_palette(10, start=0.5, rot=-0.75, dark=0.15, light=0.85, reverse=True)
colorset25 = sns.cubehelix_palette(25, start=0.5, rot=-0.75, dark=0.15, light=0.85, reverse=True)
colormap = sns.cubehelix_palette(8, start=0.5, rot=-0.75, dark=0.15, light=0.85, reverse=True, as_cmap=True)
palette1 = sns.color_palette(colorset10)
print(sns.palplot(palette1))

rcVizParams = {
    # setting label sizes
    'legend.fontsize': 12,
    'figure.figsize': (12,6),
    'axes.labelsize': 15,
    'axes.titlesize': 18,
    'figure.titleweight': 'bold',
    'xtick.labelsize': 12,
    'ytick.labelsize': 12,
    'figure.dpi': 120,
    # setting colors
    'text.color': 'slategrey',
    'axes.labelcolor': 'slategrey',
    'xtick.color': 'slategrey',
    'ytick.color': 'slategrey',
    #'font.family': ['Helvetica'],
    "lines.linewidth": 1
}
mpl.rcParams.update(rcVizParams)
mpl.rc('axes', facecolor='#f2f4f7', edgecolor='none', axisbelow=True, grid=True)
mpl.rc('grid', color='w', linestyle='solid')
plt.style.use(u'fast')

In [4]:
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
spark.sparkContext._conf.getAll()
conf = spark.sparkContext._conf.setAll([('spark.executor.memory', '16g'), 
                     ('spark.app.name', 'Homework'), 
                     ('spark.executor.cores', '4'), 
                     ('spark.cores.max', '4'), 
                     ('spark.driver.memory','16g')])
spark = SparkSession.builder.config(conf=conf).getOrCreate()

In [5]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark import SparkConf
from pyspark.sql.functions import isnan, when, count, col
from pyspark.sql.functions import mean, max, min
import pyspark.sql.functions as func

spark = SparkSession.builder.getOrCreate()
sqlContext = SQLContext(spark)
print(spark.sql("Select 'spark' as hello ").show())

In [6]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark import SparkConf

spark = SparkSession.builder.getOrCreate()
conf = spark.sparkContext._conf.setAll([
        ('spark.executor.memory', '24g'),                          
        ('spark.app.name', 'Homework'),
        ('spark.executor.cores', '4'),
        ('spark.cores.max', '4'),
        ('spark.driver.memory', '24g')                              
])
spark = SparkSession.builder.config(conf=conf).getOrCreate()
print(spark.sql("Select 'spark' as hello ").show())
sqlContext = SQLContext(spark)
display('spark', 'spark', spark.sparkContext._conf.getAll(), 'sqlcontext', sqlContext)

In [7]:
df_oct = spark.read.csv('../input/ecommerce-behavior-data-from-multi-category-store/2019-Oct.csv', inferSchema=True, header=True, nullValue='NA')
df_oct.show(5)

In [8]:
df_nov = spark.read.csv('../input/ecommerce-behavior-data-from-multi-category-store/2019-Nov.csv', inferSchema=True, header=True, nullValue='NA')
df_nov.show(5)

In [9]:
df_dec = spark.read.csv('../input/ecommerce-events-history-in-cosmetics-shop/2019-Dec.csv', inferSchema=True, header=True, nullValue='NA')
df_dec.show(5)

In [10]:
df_jan = spark.read.csv('../input/ecommerce-events-history-in-cosmetics-shop/2020-Jan.csv', inferSchema=True, header=True, nullValue='NA')
df_jan.show(5)

In [11]:
def recordCountDfCols(dataframe):
    cols = []
    for i in dataframe.columns:
        cols.append(i)
    cols = pd.DataFrame(cols)
    record_count = dataframe.count()
    print("Total columns are: {} \nTotal records are: {} \nThe column names are: \n{}".format(len(cols), record_count, cols))
    return (record_count)

In [12]:
oct_record_count = recordCountDfCols(df_oct)

In [13]:
nov_record_count = recordCountDfCols(df_nov)

In [14]:
dec_record_count = recordCountDfCols(df_dec)

In [15]:
jan_record_count = recordCountDfCols(df_jan)

In [16]:
def nullCountPercent(dataframe, record_count):
    df_null = dataframe.select([count(when(col(c).isNull(), c)).alias(c) for c in dataframe.columns])
    display(print('-'*5, 'NULL VALUE COUNTS', '-'*5, 'NULL VALUE PERCENTS', '-'*5), df_null.toPandas(), ((df_null.toPandas()/record_count)*100).round(2))

In [17]:
nullCountPercent(df_oct, oct_record_count)

In [18]:
nullCountPercent(df_nov, nov_record_count)

In [19]:
nullCountPercent(df_dec, dec_record_count)

In [20]:
nullCountPercent(df_jan, jan_record_count)

In [21]:
import pyspark.sql.functions as sparkf
from pyspark.sql.functions import col
from pyspark.sql.functions import from_unixtime, date_format, to_date
from pyspark.sql.functions import to_timestamp, unix_timestamp, hour, minute
from pyspark.sql.functions import split

def transformDatesColValues(dataframe):
    dataframe = dataframe.withColumn(
        "event_time",
        to_timestamp(dataframe['event_time'], 'yyyy-MM-dd HH:mm:ss'))

    dataframe = dataframe.withColumn('event_totd', date_format('event_time', 'HH:mm:ss')) \
                    .withColumn('event_date', date_format('event_time', 'MM-dd-yyyy'))
    dataframe = dataframe.withColumn("event_date", to_date(dataframe['event_time'], 'yyyy-MM-dd'))\
                    .withColumn("event_hour", hour(dataframe['event_time'])+1)\
                    .withColumn("event_minute", minute(dataframe['event_time']))
    dataframe = dataframe.withColumn( 'Time1440Mins',
        (dataframe['event_hour'] * 60 + dataframe['event_minute']))
    dataframe = dataframe.withColumn('dotw_num',  date_format('event_time', 'u'))\
                              .withColumn('dotw_day', date_format('event_time', 'E'))\
                              .withColumn('dotm_num', date_format('event_time', 'd'))\
                              .withColumn('woty_num', date_format('event_time', 'w'))
    dataframe = dataframe.withColumn('dotw_num',  dataframe['dotw_num'].cast('integer'))\
                    .withColumn('dotm_num',  dataframe['dotm_num'].cast('integer'))\
                    .withColumn('woty_num',  dataframe['woty_num'].cast('integer'))
    dataframe = dataframe.withColumn('primary_cat', dataframe['category_code']) \
                               .withColumn('primary_cat', split(dataframe['category_code'], "\\.").getItem(0))
    dataframe = dataframe.withColumn('product_id',  dataframe['product_id'].cast('string'))\
                                .withColumn('category_id',  dataframe['category_id'].cast('string'))
    dataframe = dataframe.withColumn('PurchaseYN', (dataframe['event_type']=='purchase').cast('integer'))
    print('Completed')
    return(dataframe)

In [22]:
df_oct = transformDatesColValues(df_oct)
display(df_oct.dtypes)

In [23]:
df_nov = transformDatesColValues(df_nov)
display(df_nov.dtypes)

In [24]:
df_dec = transformDatesColValues(df_dec)
display(df_dec.dtypes)

In [25]:
df_jan = transformDatesColValues(df_jan)
display(df_jan.dtypes)

In [26]:
df_oct.createOrReplaceTempView("oct_table")
df_oct.filter('PurchaseYN == 1').createOrReplaceTempView('oct_purchased')

In [27]:
df_nov.createOrReplaceTempView("nov_table")
df_nov.filter('PurchaseYN == 1').createOrReplaceTempView('nov_purchased')

In [28]:
df_dec.createOrReplaceTempView("dec_table")
df_dec.filter('PurchaseYN == 1').createOrReplaceTempView('dec_purchased')

In [29]:
df_jan.createOrReplaceTempView("jan_table")
df_jan.filter('PurchaseYN == 1').createOrReplaceTempView('jan_purchased')

In [30]:
spark.sql('''
select primary_cat as Category, round(avg(price),2) as AvgPrice
from oct_table
group by Category
order by AvgPrice DESC
limit 20''').show()

spark.sql('''
select primary_cat as Category, round(min(price),2) as MinPrice
from oct_table
group by Category
order by MinPrice ASC
limit 20''').show()

spark.sql('''
select primary_cat as Category, round(max(price),2) as MaxPrice
from oct_table
group by Category
order by MaxPrice DESC
limit 20''').show()

In [31]:
spark.sql('''
select primary_cat as Category, round(avg(price),2) as AvgPrice
from nov_table
group by Category
order by AvgPrice DESC
limit 20''').show()

spark.sql('''
select primary_cat as Category, round(min(price),2) as MinPrice
from nov_table
group by Category
order by MinPrice ASC
limit 20''').show()

spark.sql('''
select primary_cat as Category, round(max(price),2) as MaxPrice
from nov_table
group by Category
order by MaxPrice DESC
limit 20''').show()

In [32]:
spark.sql('''
select brand as Brand, round(avg(price),2) as AvgPrice
from oct_table
group by brand
order by AvgPrice DESC
limit 20''').show()

spark.sql('''
select brand as Brand, round(min(price),2) as MinPrice
from oct_table
group by brand
order by MinPrice ASC
limit 20''').show()

spark.sql('''
select brand as Brand, round(max(price),2) as MaxPrice
from oct_table
group by brand
order by MaxPrice DESC
limit 20''').show()

In [33]:
spark.sql('''
select brand as Brand, round(avg(price),2) as AvgPrice
from nov_table
group by brand
order by AvgPrice DESC
limit 20''').show()

spark.sql('''
select brand as Brand, round(min(price),2) as MinPrice
from nov_table
group by brand
order by MinPrice ASC
limit 20''').show()

spark.sql('''
select brand as Brand, round(max(price),2) as MaxPrice
from nov_table
group by brand
order by MaxPrice DESC
limit 20''').show()

In [34]:
spark.sql('''
select primary_cat as Category, count(primary_cat) as Popularity
from oct_table
group by Category
order by Popularity DESC
limit 20''').show()

spark.sql('''
select brand as Brand, count(brand) as Popularity
from oct_table
group by Brand
order by Popularity DESC
limit 20''').show()

spark.sql('''
select user_id as User, count(user_id) as Popularity
from oct_table
group by User
order by Popularity DESC
limit 20''').show()

In [35]:
spark.sql('''
select primary_cat as Category, count(primary_cat) as Popularity
from nov_table
group by Category
order by Popularity DESC
limit 20''').show()

spark.sql('''
select brand as Brand, count(brand) as Popularity
from nov_table
group by Brand
order by Popularity DESC
limit 20''').show()

spark.sql('''
select user_id as User, count(user_id) as Popularity
from nov_table
group by User
order by Popularity DESC
limit 20''').show()

In [36]:
spark.sql('''
SELECT event_type as Event, 
    round(count(price),2) as Count,
    round(mean(price),2) as Mean,
    round(std(price),2) as Std,
    round(min(price),2) as Min,
    round(max(price),2) as Max
FROM oct_table
group by Event
''').show()

In [37]:
spark.sql('''
SELECT event_type as Event, 
    round(count(price),2) as Count,
    round(mean(price),2) as Mean,
    round(std(price),2) as Std,
    round(min(price),2) as Min,
    round(max(price),2) as Max
FROM nov_table
group by Event
''').show()

In [38]:
df_oct.groupby('primary_cat')\
        .pivot('event_type')\
        .agg(func.round(mean('price'), 2))\
        .sort('primary_cat').show(20)

df_oct.groupby('brand')\
        .pivot('event_type')\
        .agg(func.round(mean('price'), 2))\
        .sort('brand').show(20)

In [39]:
df_nov.groupby('primary_cat')\
        .pivot('event_type')\
        .agg(func.round(mean('price'), 2))\
        .sort('primary_cat').show(20)

df_nov.groupby('brand')\
        .pivot('event_type')\
        .agg(func.round(mean('price'), 2))\
        .sort('brand').show(20)

In [40]:
# Revenue per product id ~10mins
topRevProducts = spark.sql('''
SELECT round(sum(price),2) as Revenue, product_id as ProductID 
FROM oct_purchased 
GROUP BY ProductID 
ORDER BY Revenue DESC 
Limit 25
''').toPandas()
topRevProductsPandas = topRevProducts.reset_index()

# Revenue per user_id
topRevUser = spark.sql('''
SELECT round(sum(price),2) as Revenue, user_id as UserID 
FROM oct_purchased 
GROUP BY UserID 
ORDER BY Revenue DESC 
Limit 25''').toPandas()
topRevUserPandas = topRevUser.reset_index()

# Revenue per brand
topRevBrandPandas = spark.sql('''
SELECT round(sum(price),2) as Revenue, brand as Brand 
FROM oct_purchased 
WHERE Brand IS NOT NULL 
GROUP BY Brand 
ORDER BY Revenue DESC 
Limit 25''').toPandas()
topRevBrandPandas = topRevBrandPandas.reset_index()

# Revenue per Category
topRevCatPandas = spark.sql('''
SELECT round(sum(price),2) as Revenue, primary_cat as Category 
FROM oct_purchased 
WHERE primary_cat IS NOT NULL 
GROUP BY Category 
ORDER BY Revenue DESC 
Limit 25''').toPandas()
topRevCatPandas = topRevCatPandas.reset_index()

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, sharex=False, sharey=False, figsize=(12,12))
order1 = list(topRevProductsPandas['ProductID'])
g1 = sns.barplot(y='ProductID', x='Revenue', orient='h', data=topRevProductsPandas, palette=colorset25, hue='index', dodge=False, alpha=0.9, ax=ax1, order=order1)

order2 = list(topRevUserPandas['UserID'])
g2 = sns.barplot(y='UserID', x='Revenue', orient='h', data=topRevUserPandas, palette=colorset25, hue='index', dodge=False, alpha=0.9, ax=ax2, order=order2)

order3 = list(topRevBrandPandas['Brand'])
g3 = sns.barplot(y='Brand', x='Revenue', data=topRevBrandPandas, palette=colorset25, estimator=np.sum, hue='index', dodge=False, alpha=0.9, ax=ax3, order=order3)

order4 = list(topRevCatPandas['Category'])
g4 = sns.barplot(y='Category', x='Revenue', data=topRevCatPandas, palette=colorset10, estimator=np.sum, hue='index', dodge=False, alpha=0.9, ax=ax4, order=order4)

g1.set_title('Revenue Per Product ID - Top 25')
g2.set_title('Revenue Per User - Top 25')
g3.set_title('Revenue Per Brand - Top 25')
g4.set_title('Revenue Per Category - Top 25')

plt.setp(g1.get_xticklabels(), rotation=30)
plt.setp(g2.get_xticklabels(), rotation=30)
plt.setp(g3.get_xticklabels(), rotation=30)
plt.setp(g4.get_xticklabels(), rotation=30)

g1.legend_.remove()
g2.legend_.remove()
g3.legend_.remove()
g4.legend_.remove()

fig.tight_layout()
plt.show()

In [41]:
# Revenue per product id ~10mins
topRevProducts = spark.sql('''
SELECT round(sum(price),2) as Revenue, product_id as ProductID 
FROM nov_purchased 
GROUP BY ProductID 
ORDER BY Revenue DESC 
Limit 25
''').toPandas()
topRevProductsPandas = topRevProducts.reset_index()

# Revenue per user_id
topRevUser = spark.sql('''
SELECT round(sum(price),2) as Revenue, user_id as UserID 
FROM nov_purchased 
GROUP BY UserID 
ORDER BY Revenue DESC 
Limit 25''').toPandas()
topRevUserPandas = topRevUser.reset_index()

# Revenue per brand
topRevBrandPandas = spark.sql('''
SELECT round(sum(price),2) as Revenue, brand as Brand 
FROM nov_purchased 
WHERE Brand IS NOT NULL 
GROUP BY Brand 
ORDER BY Revenue DESC 
Limit 25''').toPandas()
topRevBrandPandas = topRevBrandPandas.reset_index()

# Revenue per Category
topRevCatPandas = spark.sql('''
SELECT round(sum(price),2) as Revenue, primary_cat as Category 
FROM nov_purchased 
WHERE primary_cat IS NOT NULL 
GROUP BY Category 
ORDER BY Revenue DESC 
Limit 25''').toPandas()
topRevCatPandas = topRevCatPandas.reset_index()

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, sharex=False, sharey=False, figsize=(12,12))
order1 = list(topRevProductsPandas['ProductID'])
g1 = sns.barplot(y='ProductID', x='Revenue', orient='h', data=topRevProductsPandas, palette=colorset25, hue='index', dodge=False, alpha=0.9, ax=ax1, order=order1)

order2 = list(topRevUserPandas['UserID'])
g2 = sns.barplot(y='UserID', x='Revenue', orient='h', data=topRevUserPandas, palette=colorset25, hue='index', dodge=False, alpha=0.9, ax=ax2, order=order2)

order3 = list(topRevBrandPandas['Brand'])
g3 = sns.barplot(y='Brand', x='Revenue', data=topRevBrandPandas, palette=colorset25, estimator=np.sum, hue='index', dodge=False, alpha=0.9, ax=ax3, order=order3)

order4 = list(topRevCatPandas['Category'])
g4 = sns.barplot(y='Category', x='Revenue', data=topRevCatPandas, palette=colorset10, estimator=np.sum, hue='index', dodge=False, alpha=0.9, ax=ax4, order=order4)

g1.set_title('Revenue Per Product ID - Top 25')
g2.set_title('Revenue Per User - Top 25')
g3.set_title('Revenue Per Brand - Top 25')
g4.set_title('Revenue Per Category - Top 25')

plt.setp(g1.get_xticklabels(), rotation=30)
plt.setp(g2.get_xticklabels(), rotation=30)
plt.setp(g3.get_xticklabels(), rotation=30)
plt.setp(g4.get_xticklabels(), rotation=30)

g1.legend_.remove()
g2.legend_.remove()
g3.legend_.remove()
g4.legend_.remove()

fig.tight_layout()
plt.show()

In [42]:
# Revenue per product id ~10mins
topRevProducts = spark.sql('''
SELECT round(sum(price),2) as Revenue, product_id as ProductID 
FROM dec_purchased 
GROUP BY ProductID 
ORDER BY Revenue DESC 
Limit 25
''').toPandas()
topRevProductsPandas = topRevProducts.reset_index()

# Revenue per user_id
topRevUser = spark.sql('''
SELECT round(sum(price),2) as Revenue, user_id as UserID 
FROM dec_purchased 
GROUP BY UserID 
ORDER BY Revenue DESC 
Limit 25''').toPandas()
topRevUserPandas = topRevUser.reset_index()

# Revenue per brand
topRevBrandPandas = spark.sql('''
SELECT round(sum(price),2) as Revenue, brand as Brand 
FROM dec_purchased 
WHERE Brand IS NOT NULL 
GROUP BY Brand 
ORDER BY Revenue DESC 
Limit 25''').toPandas()
topRevBrandPandas = topRevBrandPandas.reset_index()

# Revenue per Category
topRevCatPandas = spark.sql('''
SELECT round(sum(price),2) as Revenue, primary_cat as Category 
FROM dec_purchased 
WHERE primary_cat IS NOT NULL 
GROUP BY Category 
ORDER BY Revenue DESC 
Limit 25''').toPandas()
topRevCatPandas = topRevCatPandas.reset_index()

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, sharex=False, sharey=False, figsize=(12,12))
order1 = list(topRevProductsPandas['ProductID'])
g1 = sns.barplot(y='ProductID', x='Revenue', orient='h', data=topRevProductsPandas, palette=colorset25, hue='index', dodge=False, alpha=0.9, ax=ax1, order=order1)

order2 = list(topRevUserPandas['UserID'])
g2 = sns.barplot(y='UserID', x='Revenue', orient='h', data=topRevUserPandas, palette=colorset25, hue='index', dodge=False, alpha=0.9, ax=ax2, order=order2)

order3 = list(topRevBrandPandas['Brand'])
g3 = sns.barplot(y='Brand', x='Revenue', data=topRevBrandPandas, palette=colorset25, estimator=np.sum, hue='index', dodge=False, alpha=0.9, ax=ax3, order=order3)

order4 = list(topRevCatPandas['Category'])
g4 = sns.barplot(y='Category', x='Revenue', data=topRevCatPandas, palette=colorset10, estimator=np.sum, hue='index', dodge=False, alpha=0.9, ax=ax4, order=order4)

g1.set_title('Revenue Per Product ID - Top 25')
g2.set_title('Revenue Per User - Top 25')
g3.set_title('Revenue Per Brand - Top 25')
g4.set_title('Revenue Per Category - Top 25')

plt.setp(g1.get_xticklabels(), rotation=30)
plt.setp(g2.get_xticklabels(), rotation=30)
plt.setp(g3.get_xticklabels(), rotation=30)
plt.setp(g4.get_xticklabels(), rotation=30)

g1.legend_.remove()
g2.legend_.remove()
g3.legend_.remove()
g4.legend_.remove()

fig.tight_layout()
plt.show()

In [43]:
# Revenue per product id ~10mins
topRevProducts = spark.sql('''
SELECT round(sum(price),2) as Revenue, product_id as ProductID 
FROM jan_purchased 
GROUP BY ProductID 
ORDER BY Revenue DESC 
Limit 25
''').toPandas()
topRevProductsPandas = topRevProducts.reset_index()

# Revenue per user_id
topRevUser = spark.sql('''
SELECT round(sum(price),2) as Revenue, user_id as UserID 
FROM jan_purchased 
GROUP BY UserID 
ORDER BY Revenue DESC 
Limit 25''').toPandas()
topRevUserPandas = topRevUser.reset_index()

# Revenue per brand
topRevBrandPandas = spark.sql('''
SELECT round(sum(price),2) as Revenue, brand as Brand 
FROM jan_purchased 
WHERE Brand IS NOT NULL 
GROUP BY Brand 
ORDER BY Revenue DESC 
Limit 25''').toPandas()
topRevBrandPandas = topRevBrandPandas.reset_index()

# Revenue per Category
topRevCatPandas = spark.sql('''
SELECT round(sum(price),2) as Revenue, primary_cat as Category 
FROM jan_purchased 
WHERE primary_cat IS NOT NULL 
GROUP BY Category 
ORDER BY Revenue DESC 
Limit 25''').toPandas()
topRevCatPandas = topRevCatPandas.reset_index()

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, sharex=False, sharey=False, figsize=(12,12))
order1 = list(topRevProductsPandas['ProductID'])
g1 = sns.barplot(y='ProductID', x='Revenue', orient='h', data=topRevProductsPandas, palette=colorset25, hue='index', dodge=False, alpha=0.9, ax=ax1, order=order1)

order2 = list(topRevUserPandas['UserID'])
g2 = sns.barplot(y='UserID', x='Revenue', orient='h', data=topRevUserPandas, palette=colorset25, hue='index', dodge=False, alpha=0.9, ax=ax2, order=order2)

order3 = list(topRevBrandPandas['Brand'])
g3 = sns.barplot(y='Brand', x='Revenue', data=topRevBrandPandas, palette=colorset25, estimator=np.sum, hue='index', dodge=False, alpha=0.9, ax=ax3, order=order3)

order4 = list(topRevCatPandas['Category'])
g4 = sns.barplot(y='Category', x='Revenue', data=topRevCatPandas, palette=colorset10, estimator=np.sum, hue='index', dodge=False, alpha=0.9, ax=ax4, order=order4)

g1.set_title('Revenue Per Product ID - Top 25')
g2.set_title('Revenue Per User - Top 25')
g3.set_title('Revenue Per Brand - Top 25')
g4.set_title('Revenue Per Category - Top 25')

plt.setp(g1.get_xticklabels(), rotation=30)
plt.setp(g2.get_xticklabels(), rotation=30)
plt.setp(g3.get_xticklabels(), rotation=30)
plt.setp(g4.get_xticklabels(), rotation=30)

g1.legend_.remove()
g2.legend_.remove()
g3.legend_.remove()
g4.legend_.remove()

fig.tight_layout()
plt.show()

In [44]:
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")
hourPurchasePerDotm = spark.sql('''
SELECT 
    round(avg(hourpurchased), 2) as AvgPurchasePerHour, 
    sum(hourpurchased) as SumPurchasePerHour, 
    event_hour as EventHour 
FROM 
    (select sum(PurchaseYN) as hourpurchased, dotm_num, event_hour 
    from oct_purchased
    group by dotm_num, event_hour)
GROUP BY event_hour
''')

hourPurchasePerDotm_sum = hourPurchasePerDotm.select(
    'EventHour', 'SumPurchasePerHour').toPandas()
hourPurchasePerDotm_avg = hourPurchasePerDotm.select(
    'EventHour', 'AvgPurchasePerHour').toPandas()

fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, sharex=False, sharey=False, gridspec_kw={'height_ratios': [4, 1, 1, 4]}, figsize=(12, 16))
g1 = sns.pointplot(x='EventHour', y='SumPurchasePerHour', data=hourPurchasePerDotm_sum, ax=ax1, markers='o', linestyles='dashdot')

g2 = sns.heatmap(pd.DataFrame(
    hourPurchasePerDotm_sum.groupby('EventHour')
    ['SumPurchasePerHour'].sum()).sort_values('SumPurchasePerHour',
                                              ascending=False).T,
                 linewidths=.2,
                 cmap=colormap,
                 cbar=False,
                 fmt='.1f',
                 annot=True,
                 yticklabels=False,
                 annot_kws={'rotation': 60},
                 alpha=0.8,
                 ax=ax2)

g3 = sns.heatmap(pd.DataFrame(
    hourPurchasePerDotm_avg.groupby('EventHour')
    ['AvgPurchasePerHour'].sum()).sort_values('AvgPurchasePerHour',
                                              ascending=False).T,
                 linewidths=.2,
                 cmap=colormap,
                 cbar=False,
                 fmt='.1f',
                 annot=True,
                 yticklabels=False,
                 annot_kws={'rotation': 60},
                 alpha=0.8,
                 ax=ax3)

g4 = sns.pointplot(x='EventHour', y='AvgPurchasePerHour', data=hourPurchasePerDotm_avg, ax=ax4, markers='o', linestyles='dashdot')

meanSum = round(hourPurchasePerDotm_sum['SumPurchasePerHour'].mean(),1)
g1.axhline(meanSum, linestyle=':', color='darkslateblue', alpha=0.5)
g1.text(0,
        meanSum,
        'Average {}'.format(meanSum),
        fontsize=10,
        weight='bold',
        va='bottom',
        ha='left',
        color='darkslateblue',
        alpha=0.5)

meanAvg = round(hourPurchasePerDotm_avg['AvgPurchasePerHour'].mean(),1)
g4.axhline(meanAvg,
           linestyle=':',
           color='lightslategrey',
           label='Avg',
           alpha=0.5)
g4.text(0,
        meanAvg,
        'Average {}'.format(meanAvg),
        fontsize=10,
        weight='bold',
        va='bottom',
        ha='left',
        color='darkslateblue',
        alpha=0.5)

g1.set_title('Purchases Per Hour for the Month - Timeline - Sum', fontsize=14, weight='bold')
g4.set_title('Purchases Per Hour for the Month - Timeline - Avg', fontsize=14, weight='bold')

# axes titles
ax1.set(ylabel="Purchases",                 xlabel="Hour of the Day")
ax2.set(ylabel='Purchases \nOrdered \nSum', xlabel=" " )
ax3.set(ylabel='Purchases \nOrdered \nAvg', xlabel=" " )
ax4.set(ylabel="Purchases",                 xlabel="Hour of the Day")

ax1.set_yticklabels(ax1.get_yticklabels(), rotation=0)
ax3.set_yticklabels(ax1.get_yticklabels(), rotation=0)

fig.tight_layout()
plt.show()

In [45]:
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")
hourPurchasePerDotm = spark.sql('''
SELECT 
    round(avg(hourpurchased), 2) as AvgPurchasePerHour, 
    sum(hourpurchased) as SumPurchasePerHour, 
    event_hour as EventHour 
FROM 
    (select sum(PurchaseYN) as hourpurchased, dotm_num, event_hour 
    from nov_purchased
    group by dotm_num, event_hour)
GROUP BY event_hour
''')

hourPurchasePerDotm_sum = hourPurchasePerDotm.select(
    'EventHour', 'SumPurchasePerHour').toPandas()
hourPurchasePerDotm_avg = hourPurchasePerDotm.select(
    'EventHour', 'AvgPurchasePerHour').toPandas()

fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, sharex=False, sharey=False, gridspec_kw={'height_ratios': [4, 1, 1, 4]}, figsize=(12, 16))
g1 = sns.pointplot(x='EventHour', y='SumPurchasePerHour', data=hourPurchasePerDotm_sum, ax=ax1, markers='o', linestyles='dashdot')

g2 = sns.heatmap(pd.DataFrame(
    hourPurchasePerDotm_sum.groupby('EventHour')
    ['SumPurchasePerHour'].sum()).sort_values('SumPurchasePerHour',
                                              ascending=False).T,
                 linewidths=.2,
                 cmap=colormap,
                 cbar=False,
                 fmt='.1f',
                 annot=True,
                 yticklabels=False,
                 annot_kws={'rotation': 60},
                 alpha=0.8,
                 ax=ax2)

g3 = sns.heatmap(pd.DataFrame(
    hourPurchasePerDotm_avg.groupby('EventHour')
    ['AvgPurchasePerHour'].sum()).sort_values('AvgPurchasePerHour',
                                              ascending=False).T,
                 linewidths=.2,
                 cmap=colormap,
                 cbar=False,
                 fmt='.1f',
                 annot=True,
                 yticklabels=False,
                 annot_kws={'rotation': 60},
                 alpha=0.8,
                 ax=ax3)

g4 = sns.pointplot(x='EventHour', y='AvgPurchasePerHour', data=hourPurchasePerDotm_avg, ax=ax4, markers='o', linestyles='dashdot')

meanSum = round(hourPurchasePerDotm_sum['SumPurchasePerHour'].mean(),1)
g1.axhline(meanSum, linestyle=':', color='darkslateblue', alpha=0.5)
g1.text(0,
        meanSum,
        'Average {}'.format(meanSum),
        fontsize=10,
        weight='bold',
        va='bottom',
        ha='left',
        color='darkslateblue',
        alpha=0.5)

meanAvg = round(hourPurchasePerDotm_avg['AvgPurchasePerHour'].mean(),1)
g4.axhline(meanAvg,
           linestyle=':',
           color='lightslategrey',
           label='Avg',
           alpha=0.5)
g4.text(0,
        meanAvg,
        'Average {}'.format(meanAvg),
        fontsize=10,
        weight='bold',
        va='bottom',
        ha='left',
        color='darkslateblue',
        alpha=0.5)

g1.set_title('Purchases Per Hour for the Month - Timeline - Sum', fontsize=14, weight='bold')
g4.set_title('Purchases Per Hour for the Month - Timeline - Avg', fontsize=14, weight='bold')

# axes titles
ax1.set(ylabel="Purchases",                 xlabel="Hour of the Day")
ax2.set(ylabel='Purchases \nOrdered \nSum', xlabel=" " )
ax3.set(ylabel='Purchases \nOrdered \nAvg', xlabel=" " )
ax4.set(ylabel="Purchases",                 xlabel="Hour of the Day")

ax1.set_yticklabels(ax1.get_yticklabels(), rotation=0)
ax3.set_yticklabels(ax1.get_yticklabels(), rotation=0)

fig.tight_layout()
plt.show()

In [46]:
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")
hourPurchasePerDotm = spark.sql('''
SELECT 
    round(avg(hourpurchased), 2) as AvgPurchasePerHour, 
    sum(hourpurchased) as SumPurchasePerHour, 
    event_hour as EventHour 
FROM 
    (select sum(PurchaseYN) as hourpurchased, dotm_num, event_hour 
    from dec_purchased
    group by dotm_num, event_hour)
GROUP BY event_hour
''')

hourPurchasePerDotm_sum = hourPurchasePerDotm.select(
    'EventHour', 'SumPurchasePerHour').toPandas()
hourPurchasePerDotm_avg = hourPurchasePerDotm.select(
    'EventHour', 'AvgPurchasePerHour').toPandas()

fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, sharex=False, sharey=False, gridspec_kw={'height_ratios': [4, 1, 1, 4]}, figsize=(12, 16))
g1 = sns.pointplot(x='EventHour', y='SumPurchasePerHour', data=hourPurchasePerDotm_sum, ax=ax1, markers='o', linestyles='dashdot')

g2 = sns.heatmap(pd.DataFrame(
    hourPurchasePerDotm_sum.groupby('EventHour')
    ['SumPurchasePerHour'].sum()).sort_values('SumPurchasePerHour',
                                              ascending=False).T,
                 linewidths=.2,
                 cmap=colormap,
                 cbar=False,
                 fmt='.1f',
                 annot=True,
                 yticklabels=False,
                 annot_kws={'rotation': 60},
                 alpha=0.8,
                 ax=ax2)

g3 = sns.heatmap(pd.DataFrame(
    hourPurchasePerDotm_avg.groupby('EventHour')
    ['AvgPurchasePerHour'].sum()).sort_values('AvgPurchasePerHour',
                                              ascending=False).T,
                 linewidths=.2,
                 cmap=colormap,
                 cbar=False,
                 fmt='.1f',
                 annot=True,
                 yticklabels=False,
                 annot_kws={'rotation': 60},
                 alpha=0.8,
                 ax=ax3)

g4 = sns.pointplot(x='EventHour', y='AvgPurchasePerHour', data=hourPurchasePerDotm_avg, ax=ax4, markers='o', linestyles='dashdot')

meanSum = round(hourPurchasePerDotm_sum['SumPurchasePerHour'].mean(),1)
g1.axhline(meanSum, linestyle=':', color='darkslateblue', alpha=0.5)
g1.text(0,
        meanSum,
        'Average {}'.format(meanSum),
        fontsize=10,
        weight='bold',
        va='bottom',
        ha='left',
        color='darkslateblue',
        alpha=0.5)

meanAvg = round(hourPurchasePerDotm_avg['AvgPurchasePerHour'].mean(),1)
g4.axhline(meanAvg,
           linestyle=':',
           color='lightslategrey',
           label='Avg',
           alpha=0.5)
g4.text(0,
        meanAvg,
        'Average {}'.format(meanAvg),
        fontsize=10,
        weight='bold',
        va='bottom',
        ha='left',
        color='darkslateblue',
        alpha=0.5)

g1.set_title('Purchases Per Hour for the Month - Timeline - Sum', fontsize=14, weight='bold')
g4.set_title('Purchases Per Hour for the Month - Timeline - Avg', fontsize=14, weight='bold')

# axes titles
ax1.set(ylabel="Purchases",                 xlabel="Hour of the Day")
ax2.set(ylabel='Purchases \nOrdered \nSum', xlabel=" " )
ax3.set(ylabel='Purchases \nOrdered \nAvg', xlabel=" " )
ax4.set(ylabel="Purchases",                 xlabel="Hour of the Day")

ax1.set_yticklabels(ax1.get_yticklabels(), rotation=0)
ax3.set_yticklabels(ax1.get_yticklabels(), rotation=0)

fig.tight_layout()
plt.show()

In [47]:
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")
hourPurchasePerDotm = spark.sql('''
SELECT 
    round(avg(hourpurchased), 2) as AvgPurchasePerHour, 
    sum(hourpurchased) as SumPurchasePerHour, 
    event_hour as EventHour 
FROM 
    (select sum(PurchaseYN) as hourpurchased, dotm_num, event_hour 
    from jan_purchased
    group by dotm_num, event_hour)
GROUP BY event_hour
''')

hourPurchasePerDotm_sum = hourPurchasePerDotm.select(
    'EventHour', 'SumPurchasePerHour').toPandas()
hourPurchasePerDotm_avg = hourPurchasePerDotm.select(
    'EventHour', 'AvgPurchasePerHour').toPandas()

fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, sharex=False, sharey=False, gridspec_kw={'height_ratios': [4, 1, 1, 4]}, figsize=(12, 16))
g1 = sns.pointplot(x='EventHour', y='SumPurchasePerHour', data=hourPurchasePerDotm_sum, ax=ax1, markers='o', linestyles='dashdot')

g2 = sns.heatmap(pd.DataFrame(
    hourPurchasePerDotm_sum.groupby('EventHour')
    ['SumPurchasePerHour'].sum()).sort_values('SumPurchasePerHour',
                                              ascending=False).T,
                 linewidths=.2,
                 cmap=colormap,
                 cbar=False,
                 fmt='.1f',
                 annot=True,
                 yticklabels=False,
                 annot_kws={'rotation': 60},
                 alpha=0.8,
                 ax=ax2)

g3 = sns.heatmap(pd.DataFrame(
    hourPurchasePerDotm_avg.groupby('EventHour')
    ['AvgPurchasePerHour'].sum()).sort_values('AvgPurchasePerHour',
                                              ascending=False).T,
                 linewidths=.2,
                 cmap=colormap,
                 cbar=False,
                 fmt='.1f',
                 annot=True,
                 yticklabels=False,
                 annot_kws={'rotation': 60},
                 alpha=0.8,
                 ax=ax3)

g4 = sns.pointplot(x='EventHour', y='AvgPurchasePerHour', data=hourPurchasePerDotm_avg, ax=ax4, markers='o', linestyles='dashdot')

meanSum = round(hourPurchasePerDotm_sum['SumPurchasePerHour'].mean(),1)
g1.axhline(meanSum, linestyle=':', color='darkslateblue', alpha=0.5)
g1.text(0,
        meanSum,
        'Average {}'.format(meanSum),
        fontsize=10,
        weight='bold',
        va='bottom',
        ha='left',
        color='darkslateblue',
        alpha=0.5)

meanAvg = round(hourPurchasePerDotm_avg['AvgPurchasePerHour'].mean(),1)
g4.axhline(meanAvg,
           linestyle=':',
           color='lightslategrey',
           label='Avg',
           alpha=0.5)
g4.text(0,
        meanAvg,
        'Average {}'.format(meanAvg),
        fontsize=10,
        weight='bold',
        va='bottom',
        ha='left',
        color='darkslateblue',
        alpha=0.5)

g1.set_title('Purchases Per Hour for the Month - Timeline - Sum', fontsize=14, weight='bold')
g4.set_title('Purchases Per Hour for the Month - Timeline - Avg', fontsize=14, weight='bold')

# axes titles
ax1.set(ylabel="Purchases",                 xlabel="Hour of the Day")
ax2.set(ylabel='Purchases \nOrdered \nSum', xlabel=" " )
ax3.set(ylabel='Purchases \nOrdered \nAvg', xlabel=" " )
ax4.set(ylabel="Purchases",                 xlabel="Hour of the Day")

ax1.set_yticklabels(ax1.get_yticklabels(), rotation=0)
ax3.set_yticklabels(ax1.get_yticklabels(), rotation=0)

fig.tight_layout()
plt.show()

In [50]:
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")
dayPurchasePerDotm = spark.sql('''
SELECT 
    round(avg(dayPurchased), 2) as AvgPurchasePerDay,
    sum(dayPurchased) as SumPurchasePerDay,
    dotm_num as DayoftheMonth
FROM 
    (select sum(PurchaseYN) as dayPurchased, dotm_num, event_hour
    from oct_purchased
    group by event_hour, dotm_num)
GROUP BY dotm_num
''')


dayPurchasePerDotm_sum = dayPurchasePerDotm.select('DayoftheMonth', 'SumPurchasePerDay').toPandas()
dayPurchasePerDotm_avg = dayPurchasePerDotm.select('DayoftheMonth', 'AvgPurchasePerDay').toPandas()

fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, sharex=False, sharey=False, gridspec_kw={'height_ratios': [4, 1, 1, 4]}, figsize=(12, 16))
g1 = sns.pointplot(x='DayoftheMonth', y='SumPurchasePerDay', data=dayPurchasePerDotm_sum, ax=ax1, markers='o', linestyles='dashdot')

g2 = sns.heatmap(pd.DataFrame(
    dayPurchasePerDotm_sum.groupby('DayoftheMonth')
    ['SumPurchasePerDay'].sum()).sort_values('SumPurchasePerDay',
                                              ascending=False).T,
                 linewidths=.2,
                 cmap=colormap,
                 cbar=False,
                 fmt='.1f',
                 annot=True,
                 yticklabels=False,
                 annot_kws={'rotation': 65},
                 alpha=0.8,
                 ax=ax2)

g3 = sns.heatmap(pd.DataFrame(
    dayPurchasePerDotm_avg.groupby('DayoftheMonth')
    ['AvgPurchasePerDay'].sum()).sort_values('AvgPurchasePerDay',
                                              ascending=False).T,
                 linewidths=.2,
                 cmap=colormap,
                 cbar=False,
                 fmt='.1f',
                 annot=True,
                 yticklabels=False,
                 annot_kws={'rotation': 65},
                 alpha=0.8,
                 ax=ax3)

g4 = sns.pointplot(x='DayoftheMonth', y='AvgPurchasePerDay', data=dayPurchasePerDotm_avg, ax=ax4, markers='o', linestyles='dashdot')

meanSum = round(dayPurchasePerDotm_sum['SumPurchasePerDay'].mean(),1)
g1.axhline(meanSum, linestyle=':', color='darkslateblue', alpha=0.5)
g1.text(0,
        meanSum,
        'Average {}'.format(meanSum),
        fontsize=10,
        weight='bold',
        va='bottom',
        ha='left',
        color='darkslateblue',
        alpha=0.5)

meanAvg = round(dayPurchasePerDotm_avg['AvgPurchasePerDay'].mean(), 1)
g4.axhline(meanAvg,
           linestyle=':',
           color='lightslategrey',
           label='Avg',
           alpha=0.5)
g4.text(0,
        meanAvg,
        'Average {}'.format(meanAvg),
        fontsize=10,
        weight='bold',
        va='bottom',
        ha='left',
        color='darkslateblue',
        alpha=0.5)

g1.set_title('Purchases Per Day for the Month - Timeline - Sum', fontsize=14, weight='bold')
g4.set_title('Purchases Per Day for the Month - Timeline - Avg', fontsize=14, weight='bold')

# axes titles
ax1.set(ylabel="Purchases",                 xlabel="Day of the Month")
ax2.set(ylabel='Purchases \nOrdered \nSum', xlabel=" " )
ax3.set(ylabel='Purchases \nOrdered \nAvg', xlabel=" " )
ax4.set(ylabel="Purchases",                 xlabel="Day of the Month")

ax1.set_yticklabels(ax1.get_yticklabels(), rotation=0)
ax3.set_yticklabels(ax1.get_yticklabels(), rotation=0)

fig.tight_layout()
plt.show()

In [51]:
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")
dayPurchasePerDotm = spark.sql('''
SELECT 
    round(avg(dayPurchased), 2) as AvgPurchasePerDay,
    sum(dayPurchased) as SumPurchasePerDay,
    dotm_num as DayoftheMonth
FROM 
    (select sum(PurchaseYN) as dayPurchased, dotm_num, event_hour
    from nov_purchased
    group by event_hour, dotm_num)
GROUP BY dotm_num
''')


dayPurchasePerDotm_sum = dayPurchasePerDotm.select('DayoftheMonth', 'SumPurchasePerDay').toPandas()
dayPurchasePerDotm_avg = dayPurchasePerDotm.select('DayoftheMonth', 'AvgPurchasePerDay').toPandas()

fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, sharex=False, sharey=False, gridspec_kw={'height_ratios': [4, 1, 1, 4]}, figsize=(12, 16))
g1 = sns.pointplot(x='DayoftheMonth', y='SumPurchasePerDay', data=dayPurchasePerDotm_sum, ax=ax1, markers='o', linestyles='dashdot')

g2 = sns.heatmap(pd.DataFrame(
    dayPurchasePerDotm_sum.groupby('DayoftheMonth')
    ['SumPurchasePerDay'].sum()).sort_values('SumPurchasePerDay',
                                              ascending=False).T,
                 linewidths=.2,
                 cmap=colormap,
                 cbar=False,
                 fmt='.1f',
                 annot=True,
                 yticklabels=False,
                 annot_kws={'rotation': 65},
                 alpha=0.8,
                 ax=ax2)

g3 = sns.heatmap(pd.DataFrame(
    dayPurchasePerDotm_avg.groupby('DayoftheMonth')
    ['AvgPurchasePerDay'].sum()).sort_values('AvgPurchasePerDay',
                                              ascending=False).T,
                 linewidths=.2,
                 cmap=colormap,
                 cbar=False,
                 fmt='.1f',
                 annot=True,
                 yticklabels=False,
                 annot_kws={'rotation': 65},
                 alpha=0.8,
                 ax=ax3)

g4 = sns.pointplot(x='DayoftheMonth', y='AvgPurchasePerDay', data=dayPurchasePerDotm_avg, ax=ax4, markers='o', linestyles='dashdot')

meanSum = round(dayPurchasePerDotm_sum['SumPurchasePerDay'].mean(), 1)
g1.axhline(meanSum, linestyle=':', color='darkslateblue', alpha=0.5)
g1.text(0,
        meanSum,
        'Average {}'.format(meanSum),
        fontsize=10,
        weight='bold',
        va='bottom',
        ha='left',
        color='darkslateblue',
        alpha=0.5)

meanAvg = round(dayPurchasePerDotm_avg['AvgPurchasePerDay'].mean(), 1)
g4.axhline(meanAvg,
           linestyle=':',
           color='lightslategrey',
           label='Avg',
           alpha=0.5)
g4.text(0,
        meanAvg,
        'Average {}'.format(meanAvg),
        fontsize=10,
        weight='bold',
        va='bottom',
        ha='left',
        color='darkslateblue',
        alpha=0.5)

g1.set_title('Purchases Per Day for the Month - Timeline - Sum', fontsize=14, weight='bold')
g4.set_title('Purchases Per Day for the Month - Timeline - Avg', fontsize=14, weight='bold')

# axes titles
ax1.set(ylabel="Purchases",                 xlabel="Day of the Month")
ax2.set(ylabel='Purchases \nOrdered \nSum', xlabel=" " )
ax3.set(ylabel='Purchases \nOrdered \nAvg', xlabel=" " )
ax4.set(ylabel="Purchases",                 xlabel="Day of the Month")

ax1.set_yticklabels(ax1.get_yticklabels(), rotation=0)
ax3.set_yticklabels(ax1.get_yticklabels(), rotation=0)

fig.tight_layout()
plt.show()

In [54]:
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")
dayPurchasePerDotm = spark.sql('''
SELECT 
    round(avg(dayPurchased), 2) as AvgPurchasePerDay,
    sum(dayPurchased) as SumPurchasePerDay,
    dotm_num as DayoftheMonth
FROM 
    (select sum(PurchaseYN) as dayPurchased, dotm_num, event_hour
    from dec_purchased
    group by event_hour, dotm_num)
GROUP BY dotm_num
''')


dayPurchasePerDotm_sum = dayPurchasePerDotm.select('DayoftheMonth', 'SumPurchasePerDay').toPandas()
dayPurchasePerDotm_avg = dayPurchasePerDotm.select('DayoftheMonth', 'AvgPurchasePerDay').toPandas()

fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, sharex=False, sharey=False, gridspec_kw={'height_ratios': [4, 1, 1, 4]}, figsize=(12, 16))
g1 = sns.pointplot(x='DayoftheMonth', y='SumPurchasePerDay', data=dayPurchasePerDotm_sum, ax=ax1, markers='o', linestyles='dashdot')

g2 = sns.heatmap(pd.DataFrame(
    dayPurchasePerDotm_sum.groupby('DayoftheMonth')
    ['SumPurchasePerDay'].sum()).sort_values('SumPurchasePerDay',
                                              ascending=False).T,
                 linewidths=.2,
                 cmap=colormap,
                 cbar=False,
                 fmt='.1f',
                 annot=True,
                 yticklabels=False,
                 annot_kws={'rotation': 65},
                 alpha=0.8,
                 ax=ax2)

g3 = sns.heatmap(pd.DataFrame(
    dayPurchasePerDotm_avg.groupby('DayoftheMonth')
    ['AvgPurchasePerDay'].sum()).sort_values('AvgPurchasePerDay',
                                              ascending=False).T,
                 linewidths=.2,
                 cmap=colormap,
                 cbar=False,
                 fmt='.1f',
                 annot=True,
                 yticklabels=False,
                 annot_kws={'rotation': 65},
                 alpha=0.8,
                 ax=ax3)

g4 = sns.pointplot(x='DayoftheMonth', y='AvgPurchasePerDay', data=dayPurchasePerDotm_avg, ax=ax4, markers='o', linestyles='dashdot')

meanSum = round(dayPurchasePerDotm_sum['SumPurchasePerDay'].mean(), 1)
g1.axhline(meanSum, linestyle=':', color='darkslateblue', alpha=0.5)
g1.text(0,
        meanSum,
        'Average {}'.format(meanSum),
        fontsize=10,
        weight='bold',
        va='bottom',
        ha='left',
        color='darkslateblue',
        alpha=0.5)

meanAvg = round(dayPurchasePerDotm_avg['AvgPurchasePerDay'].mean(), 1)
g4.axhline(meanAvg,
           linestyle=':',
           color='lightslategrey',
           label='Avg',
           alpha=0.5)
g4.text(0,
        meanAvg,
        'Average {}'.format(meanAvg),
        fontsize=10,
        weight='bold',
        va='bottom',
        ha='left',
        color='darkslateblue',
        alpha=0.5)

g1.set_title('Purchases Per Day for the Month - Timeline - Sum', fontsize=14, weight='bold')
g4.set_title('Purchases Per Day for the Month - Timeline - Avg', fontsize=14, weight='bold')

# axes titles
ax1.set(ylabel="Purchases",                 xlabel="Day of the Month")
ax2.set(ylabel='Purchases \nOrdered \nSum', xlabel=" " )
ax3.set(ylabel='Purchases \nOrdered \nAvg', xlabel=" " )
ax4.set(ylabel="Purchases",                 xlabel="Day of the Month")

ax1.set_yticklabels(ax1.get_yticklabels(), rotation=0)
ax3.set_yticklabels(ax1.get_yticklabels(), rotation=0)

fig.tight_layout()
plt.show()

In [55]:
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")
dayPurchasePerDotm = spark.sql('''
SELECT 
    round(avg(dayPurchased), 2) as AvgPurchasePerDay,
    sum(dayPurchased) as SumPurchasePerDay,
    dotm_num as DayoftheMonth
FROM 
    (select sum(PurchaseYN) as dayPurchased, dotm_num, event_hour
    from jan_purchased
    group by event_hour, dotm_num)
GROUP BY dotm_num
''')

dayPurchasePerDotm_sum = dayPurchasePerDotm.select('DayoftheMonth', 'SumPurchasePerDay').toPandas()
dayPurchasePerDotm_avg = dayPurchasePerDotm.select('DayoftheMonth', 'AvgPurchasePerDay').toPandas()

fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, sharex=False, sharey=False, gridspec_kw={'height_ratios': [4, 1, 1, 4]}, figsize=(12, 16))
g1 = sns.pointplot(x='DayoftheMonth', y='SumPurchasePerDay', data=dayPurchasePerDotm_sum, ax=ax1, markers='o', linestyles='dashdot')

g2 = sns.heatmap(pd.DataFrame(
    dayPurchasePerDotm_sum.groupby('DayoftheMonth')
    ['SumPurchasePerDay'].sum()).sort_values('SumPurchasePerDay',
                                              ascending=False).T,
                 linewidths=.2,
                 cmap=colormap,
                 cbar=False,
                 fmt='.1f',
                 annot=True,
                 yticklabels=False,
                 annot_kws={'rotation': 65},
                 alpha=0.8,
                 ax=ax2)

g3 = sns.heatmap(pd.DataFrame(
    dayPurchasePerDotm_avg.groupby('DayoftheMonth')
    ['AvgPurchasePerDay'].sum()).sort_values('AvgPurchasePerDay',
                                              ascending=False).T,
                 linewidths=.2,
                 cmap=colormap,
                 cbar=False,
                 fmt='.1f',
                 annot=True,
                 yticklabels=False,
                 annot_kws={'rotation': 65},
                 alpha=0.8,
                 ax=ax3)

g4 = sns.pointplot(x='DayoftheMonth', y='AvgPurchasePerDay', data=dayPurchasePerDotm_avg, ax=ax4, markers='o', linestyles='dashdot')

meanSum = round(dayPurchasePerDotm_sum['SumPurchasePerDay'].mean(), 1)
g1.axhline(meanSum, linestyle=':', color='darkslateblue', alpha=0.5)
g1.text(0,
        meanSum,
        'Average {}'.format(meanSum),
        fontsize=10,
        weight='bold',
        va='bottom',
        ha='left',
        color='darkslateblue',
        alpha=0.5)

meanAvg = round(dayPurchasePerDotm_avg['AvgPurchasePerWkDay'].mean(), 1)
g4.axhline(meanAvg,
           linestyle=':',
           color='lightslategrey',
           label='Avg',
           alpha=0.5)
g4.text(0,
        meanAvg,
        'Average {}'.format(meanAvg),
        fontsize=10,
        weight='bold',
        va='bottom',
        ha='left',
        color='darkslateblue',
        alpha=0.5)

g1.set_title('Purchases Per Day for the Month - Timeline - Sum', fontsize=14, weight='bold')
g4.set_title('Purchases Per Day for the Month - Timeline - Avg', fontsize=14, weight='bold')

# axes titles
ax1.set(ylabel="Purchases",                 xlabel="Day of the Month")
ax2.set(ylabel='Purchases \nOrdered \nSum', xlabel=" " )
ax3.set(ylabel='Purchases \nOrdered \nAvg', xlabel=" " )
ax4.set(ylabel="Purchases",                 xlabel="Day of the Month")

ax1.set_yticklabels(ax1.get_yticklabels(), rotation=0)
ax3.set_yticklabels(ax1.get_yticklabels(), rotation=0)

fig.tight_layout()
plt.show()

In [58]:
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")
wkDayPurchasePerWotM = spark.sql('''
SELECT 
    round(avg(WkDayPurchased), 2) as AvgPurchasePerWkDay, 
    sum(WkDayPurchased) as SumPurchasePerWkDay, 
    dotw_day as WkDayoftheMonth
FROM 
    (select sum(PurchaseYN) as WkDayPurchased, dotw_day, woty_num
    from oct_purchased
    group by dotw_day, woty_num)
GROUP BY dotw_day
''')

wkDayPurchasePerWotM_sum = wkDayPurchasePerWotM.select('WkDayoftheMonth', 'SumPurchasePerWkDay').toPandas()
wkDayPurchasePerWotM_avg = wkDayPurchasePerWotM.select('WkDayoftheMonth', 'AvgPurchasePerWkDay').toPandas()

fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, sharex=False, sharey=False, gridspec_kw={'height_ratios': [4, 1, 1, 4]}, figsize=(12, 16))
g1 = sns.pointplot(x='WkDayoftheMonth',
                   y='SumPurchasePerWkDay',
                   data=wkDayPurchasePerWotM_sum,
                   order=['Mon','Tue', 'Wed', 'Thu','Fri', 'Sat', 'Sun'],
                   ax=ax1,
                   markers='o',
                   linestyles='dashdot')

g2 = sns.heatmap(pd.DataFrame(
    wkDayPurchasePerWotM_sum.groupby('WkDayoftheMonth')
    ['SumPurchasePerWkDay'].sum()).sort_values('SumPurchasePerWkDay',
                                              ascending=False).T,
                 linewidths=.2,
                 cmap=colormap,
                 cbar=False,
                 fmt='.1f',
                 annot=True,
                 yticklabels=False,
                 annot_kws={'rotation': 70},
                 alpha=0.8,
                 ax=ax2)

g3 = sns.heatmap(pd.DataFrame(
    wkDayPurchasePerWotM_avg.groupby('WkDayoftheMonth')
    ['AvgPurchasePerWkDay'].sum()).sort_values('AvgPurchasePerWkDay',
                                              ascending=False).T,
                 linewidths=.2,
                 cmap=colormap,
                 cbar=False,
                 fmt='.1f',
                 annot=True,
                 yticklabels=False,
                 annot_kws={'rotation': 70},
                 alpha=0.8,
                 ax=ax3)

g4 = sns.pointplot(x='WkDayoftheMonth',
                   y='AvgPurchasePerWkDay',
                   data=wkDayPurchasePerWotM_avg,
                   order=['Mon','Tue', 'Wed', 'Thu','Fri', 'Sat', 'Sun'],
                   ax=ax4,
                   markers='o',
                   linestyles='dashdot')

meanSum = wkDayPurchasePerWotM_sum['SumPurchasePerWkDay'].mean()
g1.axhline(meanSum, linestyle=':', color='darkslateblue', alpha=0.5)
g1.text(0,
        meanSum,
        'Average {}'.format(round(meanSum,1)),
        fontsize=10,
        weight='bold',
        va='bottom',
        ha='center',
        color='darkslateblue',
        alpha=0.5)

meanAvg = wkDayPurchasePerWotM_avg['AvgPurchasePerWkDay'].mean()
g4.axhline(meanAvg,
           linestyle=':',
           color='lightslategrey',
           label='Avg',
           alpha=0.5)
g4.text(0,
        meanAvg,
        'Average {}'.format(round(meanAvg,1)),
        fontsize=10,
        weight='bold',
        va='bottom',
        ha='center',
        color='darkslateblue',
        alpha=0.5)

g1.set_title('Purchases Per Day for the Month - Timeline - Sum', fontsize=14, weight='bold')
g4.set_title('Purchases Per Day for the Month - Timeline - Avg', fontsize=14, weight='bold')

# axes titles
ax1.set(ylabel="Purchases",                 xlabel="Week Day")
ax2.set(ylabel='Purchases \nOrdered \nSum', xlabel=" " )
ax3.set(ylabel='Purchases \nOrdered \nAvg', xlabel=" " )
ax4.set(ylabel="Purchases",                 xlabel="Week Day")

ax1.set_yticklabels(ax1.get_yticklabels(), rotation=0)
ax3.set_yticklabels(ax1.get_yticklabels(), rotation=0)

fig.tight_layout()
plt.show()

In [59]:
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")
wkDayPurchasePerWotM = spark.sql('''
SELECT 
    round(avg(WkDayPurchased), 2) as AvgPurchasePerWkDay, 
    sum(WkDayPurchased) as SumPurchasePerWkDay, 
    dotw_day as WkDayoftheMonth
FROM 
    (select sum(PurchaseYN) as WkDayPurchased, dotw_day, woty_num
    from nov_purchased
    group by dotw_day, woty_num)
GROUP BY dotw_day
''')

wkDayPurchasePerWotM_sum = wkDayPurchasePerWotM.select('WkDayoftheMonth', 'SumPurchasePerWkDay').toPandas()
wkDayPurchasePerWotM_avg = wkDayPurchasePerWotM.select('WkDayoftheMonth', 'AvgPurchasePerWkDay').toPandas()

fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, sharex=False, sharey=False, gridspec_kw={'height_ratios': [4, 1, 1, 4]}, figsize=(12, 16))
g1 = sns.pointplot(x='WkDayoftheMonth',
                   y='SumPurchasePerWkDay',
                   data=wkDayPurchasePerWotM_sum,
                   order=['Mon','Tue', 'Wed', 'Thu','Fri', 'Sat', 'Sun'],
                   ax=ax1,
                   markers='o',
                   linestyles='dashdot')

g2 = sns.heatmap(pd.DataFrame(
    wkDayPurchasePerWotM_sum.groupby('WkDayoftheMonth')
    ['SumPurchasePerWkDay'].sum()).sort_values('SumPurchasePerWkDay',
                                              ascending=False).T,
                 linewidths=.2,
                 cmap=colormap,
                 cbar=False,
                 fmt='.1f',
                 annot=True,
                 yticklabels=False,
                 annot_kws={'rotation': 70},
                 alpha=0.8,
                 ax=ax2)

g3 = sns.heatmap(pd.DataFrame(
    wkDayPurchasePerWotM_avg.groupby('WkDayoftheMonth')
    ['AvgPurchasePerWkDay'].sum()).sort_values('AvgPurchasePerWkDay',
                                              ascending=False).T,
                 linewidths=.2,
                 cmap=colormap,
                 cbar=False,
                 fmt='.1f',
                 annot=True,
                 yticklabels=False,
                 annot_kws={'rotation': 70},
                 alpha=0.8,
                 ax=ax3)

g4 = sns.pointplot(x='WkDayoftheMonth',
                   y='AvgPurchasePerWkDay',
                   data=wkDayPurchasePerWotM_avg,
                   order=['Mon','Tue', 'Wed', 'Thu','Fri', 'Sat', 'Sun'],
                   ax=ax4,
                   markers='o',
                   linestyles='dashdot')

meanSum = wkDayPurchasePerWotM_sum['SumPurchasePerWkDay'].mean()
g1.axhline(meanSum, linestyle=':', color='darkslateblue', alpha=0.5)
g1.text(0,
        meanSum,
        'Average {}'.format(round(meanSum,1)),
        fontsize=10,
        weight='bold',
        va='bottom',
        ha='center',
        color='darkslateblue',
        alpha=0.5)

meanAvg = wkDayPurchasePerWotM_avg['AvgPurchasePerWkDay'].mean()
g4.axhline(meanAvg,
           linestyle=':',
           color='lightslategrey',
           label='Avg',
           alpha=0.5)
g4.text(0,
        meanAvg,
        'Average {}'.format(round(meanAvg,1)),
        fontsize=10,
        weight='bold',
        va='bottom',
        ha='center',
        color='darkslateblue',
        alpha=0.5)

g1.set_title('Purchases Per Day for the Month - Timeline - Sum', fontsize=14, weight='bold')
g4.set_title('Purchases Per Day for the Month - Timeline - Avg', fontsize=14, weight='bold')

# axes titles
ax1.set(ylabel="Purchases",                 xlabel="Week Day")
ax2.set(ylabel='Purchases \nOrdered \nSum', xlabel=" " )
ax3.set(ylabel='Purchases \nOrdered \nAvg', xlabel=" " )
ax4.set(ylabel="Purchases",                 xlabel="Week Day")

ax1.set_yticklabels(ax1.get_yticklabels(), rotation=0)
ax3.set_yticklabels(ax1.get_yticklabels(), rotation=0)

fig.tight_layout()
plt.show()

In [60]:
wkDayPurchasePerWotM = spark.sql('''
SELECT 
    round(avg(WkDayPurchased), 2) as AvgPurchasePerWkDay, 
    sum(WkDayPurchased) as SumPurchasePerWkDay, 
    dotw_day as WkDayoftheMonth
FROM 
    (select sum(PurchaseYN) as WkDayPurchased, dotw_day, woty_num
    from dec_purchased
    group by dotw_day, woty_num)
GROUP BY dotw_day
''')

wkDayPurchasePerWotM_sum = wkDayPurchasePerWotM.select('WkDayoftheMonth', 'SumPurchasePerWkDay').toPandas()
wkDayPurchasePerWotM_avg = wkDayPurchasePerWotM.select('WkDayoftheMonth', 'AvgPurchasePerWkDay').toPandas()

fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, sharex=False, sharey=False, gridspec_kw={'height_ratios': [4, 1, 1, 4]}, figsize=(12, 16))
g1 = sns.pointplot(x='WkDayoftheMonth',
                   y='SumPurchasePerWkDay',
                   data=wkDayPurchasePerWotM_sum,
                   order=['Mon','Tue', 'Wed', 'Thu','Fri', 'Sat', 'Sun'],
                   ax=ax1,
                   markers='o',
                   linestyles='dashdot')

g2 = sns.heatmap(pd.DataFrame(
    wkDayPurchasePerWotM_sum.groupby('WkDayoftheMonth')
    ['SumPurchasePerWkDay'].sum()).sort_values('SumPurchasePerWkDay',
                                              ascending=False).T,
                 linewidths=.2,
                 cmap=colormap,
                 cbar=False,
                 fmt='.1f',
                 annot=True,
                 yticklabels=False,
                 annot_kws={'rotation': 70},
                 alpha=0.8,
                 ax=ax2)

g3 = sns.heatmap(pd.DataFrame(
    wkDayPurchasePerWotM_avg.groupby('WkDayoftheMonth')
    ['AvgPurchasePerWkDay'].sum()).sort_values('AvgPurchasePerWkDay',
                                              ascending=False).T,
                 linewidths=.2,
                 cmap=colormap,
                 cbar=False,
                 fmt='.1f',
                 annot=True,
                 yticklabels=False,
                 annot_kws={'rotation': 70},
                 alpha=0.8,
                 ax=ax3)

g4 = sns.pointplot(x='WkDayoftheMonth',
                   y='AvgPurchasePerWkDay',
                   data=wkDayPurchasePerWotM_avg,
                   order=['Mon','Tue', 'Wed', 'Thu','Fri', 'Sat', 'Sun'],
                   ax=ax4,
                   markers='o',
                   linestyles='dashdot')

meanSum = wkDayPurchasePerWotM_sum['SumPurchasePerWkDay'].mean()
g1.axhline(meanSum, linestyle=':', color='darkslateblue', alpha=0.5)
g1.text(0,
        meanSum,
        'Average {}'.format(round(meanSum,1)),
        fontsize=10,
        weight='bold',
        va='bottom',
        ha='center',
        color='darkslateblue',
        alpha=0.5)

meanAvg = wkDayPurchasePerWotM_avg['AvgPurchasePerWkDay'].mean()
g4.axhline(meanAvg,
           linestyle=':',
           color='lightslategrey',
           label='Avg',
           alpha=0.5)
g4.text(0,
        meanAvg,
        'Average {}'.format(round(meanAvg,1)),
        fontsize=10,
        weight='bold',
        va='bottom',
        ha='center',
        color='darkslateblue',
        alpha=0.5)

g1.set_title('Purchases Per Day for the Month - Timeline - Sum', fontsize=14, weight='bold')
g4.set_title('Purchases Per Day for the Month - Timeline - Avg', fontsize=14, weight='bold')

# axes titles
ax1.set(ylabel="Purchases",                 xlabel="Week Day")
ax2.set(ylabel='Purchases \nOrdered \nSum', xlabel=" " )
ax3.set(ylabel='Purchases \nOrdered \nAvg', xlabel=" " )
ax4.set(ylabel="Purchases",                 xlabel="Week Day")

ax1.set_yticklabels(ax1.get_yticklabels(), rotation=0)
ax3.set_yticklabels(ax1.get_yticklabels(), rotation=0)

fig.tight_layout()
plt.show()

In [61]:
wkDayPurchasePerWotM = spark.sql('''
SELECT 
    round(avg(WkDayPurchased), 2) as AvgPurchasePerWkDay, 
    sum(WkDayPurchased) as SumPurchasePerWkDay, 
    dotw_day as WkDayoftheMonth
FROM 
    (select sum(PurchaseYN) as WkDayPurchased, dotw_day, woty_num
    from jan_purchased
    group by dotw_day, woty_num)
GROUP BY dotw_day
''')

wkDayPurchasePerWotM_sum = wkDayPurchasePerWotM.select('WkDayoftheMonth', 'SumPurchasePerWkDay').toPandas()
wkDayPurchasePerWotM_avg = wkDayPurchasePerWotM.select('WkDayoftheMonth', 'AvgPurchasePerWkDay').toPandas()

fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, sharex=False, sharey=False, gridspec_kw={'height_ratios': [4, 1, 1, 4]}, figsize=(12, 16))
g1 = sns.pointplot(x='WkDayoftheMonth',
                   y='SumPurchasePerWkDay',
                   data=wkDayPurchasePerWotM_sum,
                   order=['Mon','Tue', 'Wed', 'Thu','Fri', 'Sat', 'Sun'],
                   ax=ax1,
                   markers='o',
                   linestyles='dashdot')

g2 = sns.heatmap(pd.DataFrame(
    wkDayPurchasePerWotM_sum.groupby('WkDayoftheMonth')
    ['SumPurchasePerWkDay'].sum()).sort_values('SumPurchasePerWkDay',
                                              ascending=False).T,
                 linewidths=.2,
                 cmap=colormap,
                 cbar=False,
                 fmt='.1f',
                 annot=True,
                 yticklabels=False,
                 annot_kws={'rotation': 70},
                 alpha=0.8,
                 ax=ax2)

g3 = sns.heatmap(pd.DataFrame(
    wkDayPurchasePerWotM_avg.groupby('WkDayoftheMonth')
    ['AvgPurchasePerWkDay'].sum()).sort_values('AvgPurchasePerWkDay',
                                              ascending=False).T,
                 linewidths=.2,
                 cmap=colormap,
                 cbar=False,
                 fmt='.1f',
                 annot=True,
                 yticklabels=False,
                 annot_kws={'rotation': 70},
                 alpha=0.8,
                 ax=ax3)

g4 = sns.pointplot(x='WkDayoftheMonth',
                   y='AvgPurchasePerWkDay',
                   data=wkDayPurchasePerWotM_avg,
                   order=['Mon','Tue', 'Wed', 'Thu','Fri', 'Sat', 'Sun'],
                   ax=ax4,
                   markers='o',
                   linestyles='dashdot')

meanSum = wkDayPurchasePerWotM_sum['SumPurchasePerWkDay'].mean()
g1.axhline(meanSum, linestyle=':', color='darkslateblue', alpha=0.5)
g1.text(0,
        meanSum,
        'Average {}'.format(round(meanSum,1)),
        fontsize=10,
        weight='bold',
        va='bottom',
        ha='center',
        color='darkslateblue',
        alpha=0.5)

meanAvg = wkDayPurchasePerWotM_avg['AvgPurchasePerWkDay'].mean()
g4.axhline(meanAvg,
           linestyle=':',
           color='lightslategrey',
           label='Avg',
           alpha=0.5)
g4.text(0,
        meanAvg,
        'Average {}'.format(round(meanAvg,1)),
        fontsize=10,
        weight='bold',
        va='bottom',
        ha='center',
        color='darkslateblue',
        alpha=0.5)

g1.set_title('Purchases Per Day for the Month - Timeline - Sum', fontsize=14, weight='bold')
g4.set_title('Purchases Per Day for the Month - Timeline - Avg', fontsize=14, weight='bold')

# axes titles
ax1.set(ylabel="Purchases",                 xlabel="Week Day")
ax2.set(ylabel='Purchases \nOrdered \nSum', xlabel=" " )
ax3.set(ylabel='Purchases \nOrdered \nAvg', xlabel=" " )
ax4.set(ylabel="Purchases",                 xlabel="Week Day")

ax1.set_yticklabels(ax1.get_yticklabels(), rotation=0)
ax3.set_yticklabels(ax1.get_yticklabels(), rotation=0)

fig.tight_layout()
plt.show()

In [65]:
pdCrosstabHouravg2 = df_oct.filter('PurchaseYN == 1')\
    .select('event_hour','dotw_num', 'dotw_day', 'dotm_num', 'woty_num','PurchaseYN')\
    .toPandas().set_index(['event_hour', 'dotw_day']).sort_index()
pdCrosstabHouravg2 = pdCrosstabHouravg2.groupby(['dotw_day','event_hour', 'woty_num'])['PurchaseYN'].count()
pdCrosstabHouravg2 = pdCrosstabHouravg2.reset_index()
pdCrosstabHouravg2.rename(columns={'dotw_day': 'Week Day'}, inplace=True)

g1 = sns.catplot(
    x='event_hour',
    y='PurchaseYN',
    row='Week Day',
    row_order=['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'],
    hue='woty_num',
    palette=palette1,
    data=pdCrosstabHouravg2,
    kind="point",
    markers='o',
    linestyles='--',
    height=6,
    aspect=2,
    alpha=0.4,
    legend_out=True,
    sharex=False,
    margin_titles=True
    )
g1.set(xlabel='Hour of the Day \n', ylabel='\nTotal Purchase Count')
g1._legend.set_title('Week of \nthe Year')
g1._legend.set_bbox_to_anchor((0.925, 0.965))
plt.tight_layout()
plt.show()

In [67]:
pdCrosstabHouravg2 = df_oct.filter('PurchaseYN == 1')\
    .select('event_hour', 'dotw_num', 'dotw_day', 'dotm_num', 'PurchaseYN')\
    .toPandas().set_index(['event_hour', 'dotw_day']).sort_index()
pdCrosstabHouravg2 = pdCrosstabHouravg2.groupby(['dotw_day','event_hour'])['PurchaseYN'].count()
pdCrosstabHouravg2 = pdCrosstabHouravg2.reset_index()

g1 = sns.catplot(x='event_hour',
                 y='PurchaseYN',
                 hue='dotw_day',
                 palette=palette1,
                 data=pdCrosstabHouravg2,
                 kind='point',
                 markers='o',
                 linestyles='--',
                 height=8,
                 aspect=2,
                 hue_order=['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'],
                 ci=0.95,
                 alpha=0.4)
g1.fig.subplots_adjust(top=0.9)
g1.fig.suptitle('Purchase Count Total by Hour by Week', fontsize=16)
g1._legend.set_title('Week Day')
g1.set(xlabel='Hour of the Day \n', ylabel='Total Purchase Count')
plt.show()

In [68]:
pdCrosstabHouravg2 = df_nov.filter('PurchaseYN == 1')\
    .select('event_hour','dotw_num', 'dotw_day', 'dotm_num', 'woty_num','PurchaseYN')\
    .toPandas().set_index(['event_hour', 'dotw_day']).sort_index()

pdCrosstabHouravg2 = pdCrosstabHouravg2.groupby(['dotw_day','event_hour', 'woty_num'])['PurchaseYN'].count()
pdCrosstabHouravg2 = pdCrosstabHouravg2.reset_index()
pdCrosstabHouravg2.rename(columns={'dotw_day': 'Week Day'}, inplace=True)

g1 = sns.catplot(x='event_hour',
                 y='PurchaseYN',
                 row='Week Day',
                 row_order=['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'],
                 hue='woty_num',
                 palette=palette1,
                 data=pdCrosstabHouravg2,
                 kind="point",
                 markers='o',
                 linestyles='--',
                 height=6,
                 aspect=2,
                 alpha=0.4,
                 legend_out=True,
                 sharex=False,
                 margin_titles=True)
g1.set(xlabel="Hour of the Day \n", ylabel='\nTotal Purchase Count')
g1._legend.set_title('Week of \nthe Year')
g1._legend.set_bbox_to_anchor((0.925,.965))

plt.tight_layout()
plt.show()

In [69]:
pdCrosstabHouravg2 = df_nov.filter('PurchaseYN == 1')\
    .select('event_hour','dotw_num', 'dotw_day', 'dotm_num', 'PurchaseYN')\
    .toPandas().set_index(['event_hour', 'dotw_day']).sort_index()

pdCrosstabHouravg2 = pdCrosstabHouravg2.groupby(['event_hour', 'dotw_day'])['PurchaseYN'].count()
pdCrosstabHouravg2 = pdCrosstabHouravg2.reset_index()

g1 = sns.catplot(
    x='event_hour',
    y='PurchaseYN',
    hue='dotw_day',
    palette=palette1,
    data=pdCrosstabHouravg2,
    kind="point",
    markers='o',
    linestyles='--',
    height=8,
    aspect=2,
    hue_order=['Mon', 'Tues', 'Wed', 'Thur', 'Fri', 'Sat', 'Sun'],
    ci=0.95,
    alpha=0.4)

g1.fig.subplots_adjust(top=0.9)
g1.fig.suptitle('Purchase Count Total by Hour by Week', fontsize=16)

g1._legend.set_title('Week Day')
g1.set(xlabel="Hour of the Day \n", ylabel='Total Purchase Count')

plt.show()

Machine Learning